In [1]:
import numpy as np
import cv2 as cv

In [2]:
video_path = "volleyball_match.mp4"
cap = cv.VideoCapture(video_path)

In [3]:
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

In [4]:
output_path = "volleyball_match_output.avi"
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [5]:
bg_subtractor = cv.createBackgroundSubtractorMOG2(detectShadows=False)

In [6]:
COLOR_RANGES = {
    "red": (np.array([144, 208, 123]), np.array([255, 255, 255])),
    "white": (np.array([0, 0, 200]), np.array([180, 40, 255])),
    "yellow": (np.array([10, 100, 100]), np.array([40, 255, 255])),
    "blue": (np.array([102, 73, 17]), np.array([147, 255, 255])),
}

In [7]:
KERNEL = np.ones((5, 5), np.uint8)
total_team1_players, total_team2_players = [], []

In [8]:
def circularity(area, perimeter):
    return (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

In [9]:
def is_valid_player(cnt):
    area = cv.contourArea(cnt)
    x, y, w, h = cv.boundingRect(cnt)
    
    aspect_ratio = w / float(h) 
    if area > 2000 and aspect_ratio < 1.2:  
        return True
    return False

In [10]:
def remove_extra_ball(frame):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv.rectangle(mask, (450, 100), (550, 150), 255, -1)
    return cv.bitwise_and(frame, frame, mask=cv.bitwise_not(mask))


In [11]:
def apply_mask(hsv, color_name, roi_mask=None, dilation_iterations=0):
    """Apply color masking and ROI filtering."""
    lower, upper = COLOR_RANGES[color_name]
    mask = cv.inRange(hsv, lower, upper)
    if roi_mask is not None:
        mask = cv.bitwise_and(mask, roi_mask)
    if dilation_iterations > 0:
        mask = cv.dilate(mask, None, iterations=dilation_iterations)
    return mask

In [12]:
def create_roi_mask(frame_shape):
    roi_white_team2 = np.zeros(frame_shape[:2], dtype=np.uint8)
    mask_roi_team2 = np.zeros(frame_shape[:2], dtype=np.uint8)

    cv.rectangle(roi_white_team2, (300, 328), (1020, 700), 255, -1)
    cv.rectangle(mask_roi_team2, (100, 200), (1200, 700), 255, -1)
    cv.rectangle(mask_roi_team2, (120, 270), (152, 460), 0, -1)
    cv.rectangle(mask_roi_team2, (1160, 270), (1200, 460), 0, -1)

    return roi_white_team2, mask_roi_team2


In [13]:
def count_valid_players(contours):
    return sum(1 for cnt in contours if is_valid_player(cnt))

In [14]:
def draw_player_bounding_boxes(frame, contours, color):
    for cnt in contours:
        if is_valid_player(cnt):
            x, y, w, h = cv.boundingRect(cnt)
            cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_to_show = frame.copy()
    frame = remove_extra_ball(frame)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    
    roi_white_team2, mask_roi_team2 = create_roi_mask(frame.shape)
    
    
    yellow_mask = apply_mask(hsv, "yellow", dilation_iterations=5)
    blue_mask = apply_mask(hsv, "blue")
    white_mask_team2 = apply_mask(hsv, "white", roi_white_team2)
    blue_mask_team2 = cv.bitwise_and(blue_mask, roi_white_team2)
    red_mask = apply_mask(hsv, "red", mask_roi_team2)

    white_player_mask = cv.bitwise_or(blue_mask_team2, white_mask_team2)

    mask_roi_blue_team1 = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv.rectangle(mask_roi_blue_team1, (200, 150), (1050, 400), 255, -1)
    blue_mask = cv.bitwise_and(blue_mask, mask_roi_blue_team1)
    blue_mask = cv.dilate(blue_mask, None, iterations=5)

   
    team1_mask = cv.bitwise_or(blue_mask, yellow_mask)
    team2_mask = cv.bitwise_or(red_mask, white_mask_team2)
    

    team2_mask = cv.morphologyEx(team2_mask, cv.MORPH_OPEN, KERNEL)
    team2_mask = cv.dilate(team2_mask, KERNEL, iterations=5)

    team1_contours, _ = cv.findContours(team1_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    team2_contours, _ = cv.findContours(team2_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    yellow_contours, _ = cv.findContours(yellow_mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    team1_players = count_valid_players(team1_contours)
    team2_players = count_valid_players(team2_contours)

    total_team1_players.append(team1_players)
    total_team2_players.append(team2_players)
    
    avg_team1 = sum(total_team1_players) / len(total_team1_players) if total_team1_players else 0
    avg_team2 = sum(total_team2_players) / len(total_team2_players) if total_team2_players else 0

    draw_player_bounding_boxes(frame_to_show, team1_contours, (0, 255, 255))  # Yellow
    draw_player_bounding_boxes(frame_to_show, team2_contours, (255, 0, 255))  # Red

    max_score, best_contour = 0, None
    for cnt in yellow_contours:
        area = cv.contourArea(cnt)
        perimeter = cv.arcLength(cnt, closed=True)
        if 400 < area < 1000:
            score = circularity(area, perimeter)
            if 0.7 < score < 1.2 and score > max_score:
                max_score = score
                best_contour = cnt

    if best_contour is not None:
        x, y, w, h = cv.boundingRect(best_contour)
        center = (x + w // 2, y + h // 2)
        cv.circle(frame_to_show, center, 10, (0, 255, 0), 3)

    cv.putText(frame_to_show, f"Team 1 Players: {team1_players} (Avg: {int(avg_team1)})", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv.putText(frame_to_show, f"Team 2 Players: {team2_players} (Avg: {int(avg_team2)})", (50, 80), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
    out.write(frame_to_show)
    cv.imshow("Final Output", frame_to_show)
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
